## **Setup**

In [ ]:
from google.colab import drive
mount_path = '/content/gdrive/'
drive.mount(mount_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/3wGrhr5R5hSyWJQSLekp9Uql3mKjwRlEdCElR5RLvIl0rBxpDrJDqIc
Mounted at /content/gdrive/


In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

#hide
!pip install utils
from utils import *

     |████████████████████████████████| 727kB 9.2MB/s 
     |████████████████████████████████| 358kB 29.7MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 61kB 8.3MB/s 
     |████████████████████████████████| 1.0MB 32.5MB/s 
     |████████████████████████████████| 92kB 11.0MB/s 
     |████████████████████████████████| 40kB 6.4MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 61kB 9.0MB/s 
     |████████████████████████████████| 2.6MB 54.6MB/s 


In [ ]:
from fastai.vision.all import *

In [ ]:
import os
import shutil
import glob
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import pytz
from zipfile import ZipFile
from tempfile import TemporaryDirectory
from PIL import Image

from sklearn.metrics import f1_score, accuracy_score

# plot options
# plt.rcParams.update({'font.size': 11})
plt.style.use('fivethirtyeight')

In [ ]:
# Set and test path to competition data files
competition_path = 'My Drive/AI For Good - AI Blitz 3/Snake/Data/'
model_path='My Drive/AI For Good - AI Blitz 3/Snake/Models/'
results_path='My Drive/AI For Good - AI Blitz 3/Snake/Results/'
filename = 'train.zip'
assert(os.path.exists(f'{mount_path}/{competition_path}/{filename}'))
print('Drive mounted correctly and data accessible')

Drive mounted correctly and data accessible


# **Functions**



### **Data loading**

In [ ]:
# custom copytree because Colab doesn't have the latest version of shutil.copytree which now contains the dirs_exist_ok flag solving this issue
# copied verbatim from https://stackoverflow.com/a/12514470/5991868
def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

# Unzip and combine images
Combine the train and validation sets from the competition organizers, we're going to use our own!


In [ ]:
# this can take a bit (maybe 30s)
# All_Data is the home directory for the images, from which training and validation splits will be taken
if not os.path.exists('/content/All_Data/'):
  os.mkdir('/content/All_Data/')
  os.mkdir('/content/All_Data/non_venomous/')
  os.mkdir('/content/All_Data/venomous/')

  for filename in ['train','val']:
    # create a temporary directory using TemporaryDirectory and context manager and unzip to there
    with TemporaryDirectory() as tmpdirname:
      with ZipFile(f'{mount_path}/{competition_path}/{filename}.zip', 'r') as zip_ref:
        zip_ref.extractall(tmpdirname)
      # copy the data to the All_Data dir
      copytree(f'{tmpdirname}/content/data/{filename}/non_venomous/','/content/All_Data/non_venomous/')
      copytree(f'{tmpdirname}/content/data/{filename}/venomous/','/content/All_Data/venomous/')

# Make preds

In [ ]:
# model_files=glob.glob(f'{os.path.join(mount_path,model_path)}*.pkl')
# print(model_files)

model_files=[
            #  '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_0917_resnet34_cv0.pkl',
            #  '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_1038_resnet34_cv1.pkl',              
            #  '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_1158_resnet34_cv2.pkl', 
            #  '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_1321_xresnet34_cv0.pkl',
            #  '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_1443_xresnet34_cv1.pkl',
            #  '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_1605_xresnet34_cv2.pkl',
            #  '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_1734_resnet50_cv0.pkl', 
            #  '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_1904_resnet50_cv1.pkl', 
            #  '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_2035_resnet50_cv2.pkl', 
            #  '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_2227_xresnet50_cv0.pkl',
            #  '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_2352_xresnet50_cv1.pkl', 
            #  '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200831_0116_xresnet50_cv2.pkl', 
            #  '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200831_0244_densenet121_cv0.pkl', 
            #  '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200831_0410_densenet121_cv1.pkl', 
            #  '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200831_0535_densenet121_cv2.pkl', 
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200831_0726_densenet201_cv0.pkl', 
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200831_0917_densenet201_cv1.pkl', 
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200831_1109_densenet201_cv2.pkl']

In [ ]:
# model_files=glob.glob(f'{os.path.join(mount_path,model_path)}*.pkl')

imgdict={'non_venomous':None,'venomous':None}
for venom in ['non_venomous','venomous']:
  imgdict[venom]=get_image_files('/content/All_Data/'+venom+os.sep)

for model in model_files:
  learn = load_learner(model)

  model_name=os.path.basename(model[:-4])
  print(model_name)
  for venom in ['non_venomous','venomous']:
    tst_dl=learn.dls.test_dl(imgdict[venom])

    # learn.epoch=0
    # preds = learn.tta(dl=tst_dl,n=2)[0]
    
    preds = learn.get_preds(dl=tst_dl)[0]

    np.save(f'{mount_path}{results_path}{model_name}_{venom}.npy', preds.numpy())

200831_0726_densenet201_cv0


200831_0917_densenet201_cv1


200831_1109_densenet201_cv2
